In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.fft import fft, ifft, fftfreq
import matplotlib
import bagpy
from bagpy import bagreader
import pandas as pd
from scipy.signal import argrelextrema
from matplotlib.colors import LogNorm
from matplotlib import colors
matplotlib.rcParams.update({'font.size': 18})

This script prepares me a spatial grid that has time, x, y of the motion.

In [3]:
# constants
c_0 = 3e8 # speed of light
microsteps = 800 # steps/rev
pitch = 5 # mm/rev
stepspermm = microsteps/pitch


# FMCW parameters
size_SAR_x = 400 # mm
size_SAR_y = 400 # mm

dy_sampling = 1 # mm
dx_motion_step = 1 # mm
num_x = size_SAR_x/dx_motion_step #
num_y = size_SAR_y/dy_sampling #
n_frms = int(1.30 * num_x * num_y)

v_testbed = 20.0 # mm/s
dt = dy_sampling/v_testbed # second

dCfg        =   {
                    "fStrt"     :   76e9,
                    "fStop"     :   78e9,
                    "TRampUp"   :   64e-6,
                    "TRampDo"   :   32e-6,
                    "TInt"      :   dt,
                    "N"         :   256,
                    "NrFrms"    :   n_frms,
                    "IniTim"    :   1000e-3,
                    "IniEve"    :   0
                }
n_ch = 1

file_exp = "43"
# output folder
#folder_output = '/home/qiwen/Github/UA_SAR/code/Analysis/RDA_2D_test_Feb/exp%s/'%file_exp
#folder_output = '/home/qiwen/Github/UA_SAR/code/Analysis/RDA_investigations/t_%ds/exp%s/'%(t_sar_actual,file_exp)
#folder_output = '/home/wq3939390/SAR_project/code/Analysis/Motion_Emulation/1Ddata/exp%s/'%file_exp
#folder_output = '/home/qiwen/Github/SAR_project/code/Analysis/motion_error_Sep2022/exp%s/'%file_exp
folder_output = '/home/wq3939390/SAR_project/code/Analysis/motion_error_Sep2022/exp%s/'%file_exp

In [4]:
# load motion data
#b = bagreader('/home/qiwen/Github/SAR_project/data/20220302/exp%s.bag'%(file_exp))
b = bagreader('/home/wq3939390/SAR_project/code/ros_code/bagfiles/20221019/exp%s.bag'%file_exp)

data_motion = b.message_by_topic("Testbed")
print("File saved: {}".format(data_motion))
df_bag = pd.read_csv(data_motion)
df_bag['y [mm]'] = df_bag['data']/stepspermm
df_bag

# load radar data
path = '/home/wq3939390/SAR_project/code/Radarbook/Python/PyAppNotes/output/20221019/'
#path = '/home/qiwen/Github/SAR_project/data/20220302/'
data_R = np.loadtxt(path + "R_exp%s.txt"%file_exp, delimiter = ',')
data_time = np.loadtxt(path + "SAR_time_exp%s.csv"%file_exp, delimiter = ',')

[INFO]  Successfully created the data folder /home/wq3939390/SAR_project/code/ros_code/bagfiles/20221019/exp43.
File saved: /home/wq3939390/SAR_project/code/ros_code/bagfiles/20221019/exp43/estbed.csv


Now let's find the initial time/position and final time/position, based radar time

In [5]:
print('Set position bounds in y direction:')
ymin = min(df_bag['y [mm]'])
ymax = max(df_bag['y [mm]'])
condition_y = (df_bag['y [mm]']==ymin) | (df_bag['y [mm]']==ymax) 
df_ybounds = df_bag[condition_y]
print(df_ybounds)

print("\nFind the initial time index, after radar has been activated:")
t0_radar = data_time[0]
idx_t0 = df_ybounds[df_ybounds['Time'] >= t0_radar].index[0]
idx_t1 = df_ybounds[df_ybounds['Time'] >= t0_radar].index[1] # index of the next bound
print("The initial index is: "+str(idx_t0))
print("The second index is: "+str(idx_t1))

print("\nFind the final time index, after radar has been activated:")
tf_radar = data_time[-1]
idx_tf = df_ybounds[df_ybounds['Time'] <= tf_radar].index[-1]
print("The final index is: "+str(idx_tf))

Set position bounds in y direction:
                Time     data  y [mm]
111     1.666229e+09      0.0     0.0
517     1.666229e+09  64960.0   406.0
923     1.666229e+09      0.0     0.0
1329    1.666229e+09  64960.0   406.0
1735    1.666229e+09      0.0     0.0
...              ...      ...     ...
165759  1.666239e+09      0.0     0.0
166165  1.666239e+09  64960.0   406.0
166571  1.666239e+09      0.0     0.0
166977  1.666239e+09  64960.0   406.0
167383  1.666239e+09      0.0     0.0

[413 rows x 3 columns]

Find the initial time index, after radar has been activated:
The initial index is: 517
The second index is: 923

Find the final time index, after radar has been activated:
The final index is: 166977


In [6]:
# select the motion data in a L x L square, where L is the size in y
L_y = int(idx_t1-idx_t0)
L_x = int((idx_tf-idx_t0)/L_y)
print("The size of the position data is: " + str(L_x) + " x " + str(L_y))
list_idx = np.arange(idx_t0, idx_t0+L_x*L_y, 1)
df_bag_selected = df_bag.iloc[list_idx]
print(df_bag_selected)
# now remove bounds, will not be necessary after I fix Arduino code to have 0.0 mm twice and 190.0 mm twice 
condition_remove_bounds = (df_bag_selected['y [mm]']!=ymin) & (df_bag_selected['y [mm]']!=ymax) 
df_bag_selected = df_bag_selected[condition_remove_bounds]
print(df_bag_selected)

The size of the position data is: 410 x 406
                Time     data  y [mm]
517     1.666229e+09  64960.0   406.0
518     1.666229e+09  64800.0   405.0
519     1.666229e+09  64640.0   404.0
520     1.666229e+09  64480.0   403.0
521     1.666229e+09  64320.0   402.0
...              ...      ...     ...
166972  1.666239e+09  64160.0   401.0
166973  1.666239e+09  64320.0   402.0
166974  1.666239e+09  64480.0   403.0
166975  1.666239e+09  64640.0   404.0
166976  1.666239e+09  64800.0   405.0

[166460 rows x 3 columns]
                Time     data  y [mm]
518     1.666229e+09  64800.0   405.0
519     1.666229e+09  64640.0   404.0
520     1.666229e+09  64480.0   403.0
521     1.666229e+09  64320.0   402.0
522     1.666229e+09  64160.0   401.0
...              ...      ...     ...
166972  1.666239e+09  64160.0   401.0
166973  1.666239e+09  64320.0   402.0
166974  1.666239e+09  64480.0   403.0
166975  1.666239e+09  64640.0   404.0
166976  1.666239e+09  64800.0   405.0

[166050 rows x 3

In [7]:
L_x*dx_motion_step

410

In [8]:
# construct time and position grid, as one array of [t,x,y]
a = np.arange(0,L_x*dx_motion_step,dx_motion_step)
x = np.tile(a,(L_y-1,1))
x = x.T.reshape(L_x*(L_y-1),1)
df_bag_selected['x [mm]'] = x
df_final = df_bag_selected.drop('data', axis= 1)
df_final.to_pickle(folder_output+"2D_grid.pkl")
df_final

,Time,y [mm],x [mm]
518,1.666229e+09,405.0,0
519,1.666229e+09,404.0,0
520,1.666229e+09,403.0,0
521,1.666229e+09,402.0,0
522,1.666229e+09,401.0,0
...,...,...,...
166972,1.666239e+09,401.0,409
166973,1.666239e+09,402.0,409
166974,1.666239e+09,403.0,409
166975,1.666239e+09,404.0,409


In [9]:
df = pd.read_pickle(folder_output+"2D_grid.pkl")
df

,Time,y [mm],x [mm]
518,1.666229e+09,405.0,0
519,1.666229e+09,404.0,0
520,1.666229e+09,403.0,0
521,1.666229e+09,402.0,0
522,1.666229e+09,401.0,0
...,...,...,...
166972,1.666239e+09,401.0,409
166973,1.666239e+09,402.0,409
166974,1.666239e+09,403.0,409
166975,1.666239e+09,404.0,409
